## Segmentação dos dados

As características normalmente são extraídas sobre pequenos segmentos de tamanho fixo dos dados, não no dado como um todo. Estes pequenos segmentos são chamados de janelas. A técnica de separar os dados em janelas, recebe o nome de janela deslizante (*sliding window*) e é uma forma de segmentação de dados. Uma boa prática, é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações da lacuna de uma janela e outra não sejam perdidas. Na imagem a seguir podemos observar um sinal EOG. Abaixo dele a representação de janelas: *W1*, *W2*, *W3*... Repare que entre as janelas, há uma sobreposição de tamanho *T*. 

Nesta base de dados, cada ensaio tem duração de 5 segundos. Se utilizadas janelas com tamanho de 250 ms, resultará em 20 janelas. Ao aplicar uma sobreposição de ~128ms, ficamos com 41 janelas de ~122ms.

Algumas características de EMG trabalham com o dado no domínio da frequência. Quando tais características são aplicadas, é necessário tranformar o dado para o domínio da frequência, utilizando o método de transformação de domínio `STFT`, do inglês *Short-time Fourier transform* (Transformada de Fourier de curto termo). O código divide o dado em segmentos, tanto no domínio do tempo quanto no domínio da frequência.

In [1]:
# Criando dados fictícios

import numpy as np
from scipy.signal import stft

shape = (60, 4, 1000)
data = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)
data

array([[[ -8.008673  ,  -0.58829546,  -9.858596  , ..., -17.889585  ,
           1.2247854 , -17.577307  ],
        [ -7.9405856 ,   6.960431  , -14.248193  , ...,  11.30431   ,
          -7.5641656 ,  -4.5305576 ],
        [-19.661911  , -13.885457  ,  23.848293  , ...,  -6.7502007 ,
           3.5568554 ,  14.410712  ],
        [  1.948189  ,   3.5041928 ,  10.999597  , ...,   6.238986  ,
         -22.061977  ,  -6.980445  ]],

       [[  5.683564  ,  11.982497  ,  -2.1020398 , ..., -10.473844  ,
           3.3903453 ,   9.426042  ],
        [ 11.419293  ,  -1.4191439 ,   1.0030504 , ...,  10.370554  ,
           1.1101235 ,  -3.3117485 ],
        [-12.956529  , -12.769783  ,   2.839849  , ...,   2.5993187 ,
           4.512013  , -13.182432  ],
        [-21.458878  , -13.739882  ,  -3.078786  , ...,  -1.0564513 ,
           4.104175  ,   1.8604648 ]],

       [[ -5.465092  ,  23.106627  ,  -6.7768335 , ...,  -2.4789748 ,
          -0.4654307 ,  -4.797864  ],
        [ -8.455839  ,  

In [2]:
step = 29
segment = 64
print('', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

# Janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=64, noverlap=32)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

 (60, 4, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (60, 4, 33, 64) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (60, 4, 33, 33) - (classes+ensaios, canais, janelas, linhas)


##### Desafio: Implementar um algoritmo para encontrar a mesma quantidade de janelas em ambos os domínios (tempo e frequência).

In [3]:
step = 1
segment = 64
chunks_time = None
chunks_freq = None



while True:
    n_win = int((data.shape[-1] - segment) / step) + 1
    ids = np.arange(n_win) * step

    # Janelas do dado no dominio do tempo
    chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

    # Janelas do dado no domínio da frequência
    _, _, chunks_freq = stft(data, fs=200, nperseg=64, noverlap=32)
    chunks_freq = np.swapaxes(chunks_freq, 2, 3)

    if chunks_time.shape[2] == chunks_freq.shape[2]:
        break
    else:
        step += 1

print("Step", step)
print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

Step 29
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (60, 4, 33, 64) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (60, 4, 33, 33) - (classes+ensaios, canais, janelas, linhas)
